In [10]:
import os, sys, glob, shutil, json
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import cv2
from PIL import Image
import numpy as np
from tqdm import tqdm, tqdm_notebook
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
from torch.utils import model_zoo
import time
import random

In [11]:
class SVHNDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
            # 设置最⻓长的字符⻓长度为4个
            lbl = np.array(self.img_label[index], dtype=np.int)
            lbl = list(lbl) + (4 - len(lbl)) * [10]
            return img, torch.from_numpy(np.array(lbl[:4]))
    def __len__(self):
        return len(self.img_path)

In [12]:
class Gauss_blur():
    def __init__(self, w, std):
        self.w = w
        self.std = std
        
 
    def __call__(self, img):
        if random.randint(0, 1)==0:
            return img
        else:
            img = np.asarray(img, np.float)
            img = cv2.GaussianBlur(img, (self.w, self.w), self.std)
            img = Image.fromarray(np.uint8(img))
            return img

class Noise():
    """Adds gaussian noise to a tensor.
    
    Example:
        >>> transforms.Compose([
        >>>     transforms.ToTensor(),
        >>>     Noise(0.1, 0.05)),
        >>> ])
    
    """
    def __init__(self, mean, stddev):
        self.mean = mean
        self.stddev = stddev

    def __call__(self, tensor):
        if random.randint(0, 1)==0:
            return tensor
        else:
            noise = torch.zeros_like(tensor).normal_(self.mean, self.stddev)
            return tensor.add_(noise) 
    
    
# def _random_colour_space(x):
#     output = x.convert("HSV")
#     return output 

# colour_transform = transforms.Lambda(lambda x: _random_colour_space(x))
# random_colour_transform = torchvision.transforms.RandomApply([colour_transform])

In [13]:
train_path = glob.glob('./data/mchar_train/*.png')
train_path.sort()
train_json = json.load(open('./data/mchar_train.json'))
train_label = [train_json[x]['label'] for x in train_json]
print(len(train_path), len(train_label))
train_loader = torch.utils.data.DataLoader(
    SVHNDataset(train_path, train_label,
                transforms.Compose([
                    transforms.Resize((70, 140)),
                    transforms.RandomCrop((60, 120)),
                    #transforms.RandomResizedCrop(size=(60, 120),scale=(0.9,1.0),ratio=(0.75,1.33),interpolation=2),
                    transforms.ColorJitter(0.5, 0.5, 0.5, 0.5),
                    transforms.RandomGrayscale(0.5),
                    transforms.RandomRotation(15),
                    #Gauss_blur(5, 10),
                    transforms.ToTensor(),
                    Noise(0, 0.05),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])),
    batch_size=256,
    shuffle=True,
    num_workers=10,
)

val_path = glob.glob('./data/mchar_val/*.png')
val_path.sort()
val_json = json.load(open('./data/mchar_val.json'))
val_label = [val_json[x]['label'] for x in val_json]
print(len(val_path), len(val_label))

val_loader = torch.utils.data.DataLoader(
    SVHNDataset(val_path, val_label,
                transforms.Compose([
                    transforms.Resize((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])),
    batch_size=256,
    shuffle=False,
    num_workers=10,
)

30000 30000
10000 10000


In [26]:
model_urls = {'resnet50': './weights/resnet50.pth',
              'resnet101': './weights/resnet101.pth',
             'best':'./weights/model.pt', 
              'best_acc':'./weights/model_acc.pt'
             }
class SVHN_Model1(nn.Module):
    def __init__(self):
        super(SVHN_Model1, self).__init__()
        model_conv = models.resnet50(pretrained=False)
        pre = torch.load(model_urls['resnet50'])
        model_conv.load_state_dict(pre)
        #print('model:', model_conv)
        model_conv.avgpool = nn.AdaptiveAvgPool2d(1)
        model_conv = nn.Sequential(*list(model_conv.children())[:-1])
        self.cnn = model_conv
        self.fc1 = nn.Linear(2048, 11)
        self.fc1_2 = nn.Linear(1024, 11)
        self.fc2 = nn.Linear(2048, 11)
        self.fc2_2 = nn.Linear(1024, 11)
        self.fc3 = nn.Linear(2048, 11)
        self.fc3_2 = nn.Linear(1024, 11)
        self.fc4 = nn.Linear(2048, 11)
        self.fc4_2 = nn.Linear(1024, 11)
    
    def forward(self, img):
        feat = self.cnn(img)
        # print(feat.shape)
        feat = feat.view(feat.shape[0], -1)
#         c1 = self.fc1_2(F.dropout(F.relu(self.fc1(feat)), 0.5))
#         c2 = self.fc2_2(F.dropout(F.relu(self.fc2(feat)), 0.5))
#         c3 = self.fc3_2(F.dropout(F.relu(self.fc3(feat)), 0.5))
#         c4 = self.fc4_2(F.dropout(F.relu(self.fc4(feat)), 0.5))
        c1 = self.fc1(feat)
        c2 = self.fc2(feat)
        c3 = self.fc3(feat)
        c4 = self.fc4(feat)
        return c1, c2, c3, c4

In [27]:
def train(train_loader, model, criterion, optimizer):
    # 切换模型为训练模式
    model.train()
    train_loss = []
    T0 = time.time()
    for i, (input, target) in enumerate(train_loader):
        if use_cuda:
            input = input.cuda()
            target = target.cuda()
        c0, c1, c2, c3 = model(input)
        target = target.long()
        loss = criterion(c0, target[:, 0]) + \
                criterion(c1, target[:, 1]) + \
                criterion(c2, target[:, 2]) + \
                criterion(c3, target[:, 3])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            TT = time.time()
            print(i, loss.item(), TT-T0)
            T0 = time.time()
        train_loss.append(loss.item())
    return np.mean(train_loss)

def validate(val_loader, model, criterion):
    # 切换模型为预测模型
    model.eval()
    val_loss = []
    # 不不记录模型梯度信息
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            if use_cuda:
                input = input.cuda()
                target = target.cuda()
            c0, c1, c2, c3 = model(input)
            target = target.long()
            loss = criterion(c0, target[:, 0]) + \
                    criterion(c1, target[:, 1]) + \
                    criterion(c2, target[:, 2]) + \
                    criterion(c3, target[:, 3]) 
            val_loss.append(loss.item())
    return np.mean(val_loss)

def predict(test_loader, model1, model2, tta=10):
    model.eval()
    test_pred_tta = True
    # TTA 次数
    for _ in range(tta):
        test_pred = []
        with torch.no_grad():
            for i, (input, target) in enumerate(test_loader):
                if use_cuda:
                    input = input.cuda()
                c0, c1, c2, c3 = model1(input)
                output = np.concatenate([
                    c0.data.cpu().numpy(),
                    c1.data.cpu().numpy(),
                    c2.data.cpu().numpy(),
                    c3.data.cpu().numpy()], axis=1)
                test_pred.append(output)
            test_pred = np.vstack(test_pred)
            if test_pred_tta is None:
                test_pred_tta = test_pred
            else:
                test_pred_tta += test_pred
    return test_pred_tta

def predict2(test_loader, model1, model2, tta=10):
    model1.eval()
    model2.eval()
    test_pred_tta = True
    # TTA 次数
    for _ in range(tta):
        test_pred = []
        with torch.no_grad():
            for i, (input, target) in enumerate(test_loader):
                if use_cuda:
                    input = input.cuda()
                c0, c1, c2, c3 = model1(input)
                output = np.concatenate([
                    c0.data.cpu().numpy(),
                    c1.data.cpu().numpy(),
                    c2.data.cpu().numpy(),
                    c3.data.cpu().numpy()], axis=1)
                c0, c1, c2, c3 = model1(input)
                output2 = np.concatenate([
                    c0.data.cpu().numpy(),
                    c1.data.cpu().numpy(),
                    c2.data.cpu().numpy(),
                    c3.data.cpu().numpy()], axis=1)
                test_pred.append(output+output2)
                
            test_pred = np.vstack(test_pred)
            if test_pred_tta is None:
                test_pred_tta = test_pred
            else:
                test_pred_tta += test_pred
        
    return test_pred_tta

In [28]:
learn_rate = 0.001
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = learn_rate * (0.1 ** (epoch // 10))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [30]:
model = SVHN_Model1()
# weights = model_urls['last']
# model.load_state_dict(torch.load(model_urls['last']))

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
# T_max=5
# lr_sch = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1)
# lr_sch.step()
best_loss = 1000.0
best_acc = 0
use_cuda = False
if use_cuda:
    model = model.cuda()
else:
    model = model
for epoch in range(40):
    adjust_learning_rate(optimizer,epoch)
    train_loss = train(train_loader, model, criterion, optimizer)
    val_loss = validate(val_loader, model, criterion)
    val_label = [''.join(map(str, x)) for x in val_loader.dataset.img_label]
    val_predict_label = predict(val_loader, model, 1)
    val_predict_label = np.vstack([
        val_predict_label[:, :11].argmax(1),
        val_predict_label[:, 11:22].argmax(1),
        val_predict_label[:, 22:33].argmax(1),
        val_predict_label[:, 33:44].argmax(1),
    ]).T
    val_label_pred = []
    for x in val_predict_label:
        val_label_pred.append(''.join(map(str, x[x!=10])))
    val_acc = np.mean(np.array(val_label_pred) == np.array(val_label))
    print('Epoch: {0}, Train loss: {1} \t Val loss: {2}'.format(epoch, train_loss, val_loss))
    print(val_acc)
    # 记录下验证集精度
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), './weights/model.pt')
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), './weights/model_acc.pt')


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/root/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/root/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 79, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/root/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 79, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/root/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 53, in default_collate
    storage = elem.storage()._new_shared(numel)
  File "/root/anaconda3/lib/python3.7/site-packages/torch/storage.py", line 132, in 

In [ ]:
model_1 = SVHN_Model1()
weights = model_urls['best']
model_1.load_state_dict(torch.load(model_urls['best']))
model_1 = model_1.cuda()
model_2 = SVHN_Model1()
weights = model_urls['best_acc']
model_2.load_state_dict(torch.load(model_urls['best_acc']))
model_2 = model_2.cuda()

test_path = glob.glob('./data/mchar_test_a/*.png')
test_path.sort()
test_label = [[1]] * len(test_path)
print(len(val_path), len(val_label))
test_loader = torch.utils.data.DataLoader(
    SVHNDataset(test_path, test_label,
                transforms.Compose([
                    transforms.Resize((64, 128)),
                    transforms.RandomCrop((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])),
    batch_size=256,
    shuffle=False,
    num_workers=10,
)

test_predict_label = predict2(test_loader, model_1, model_2, 10)
test_label = [''.join(map(str, x)) for x in test_loader.dataset.img_label]
test_predict_label = np.vstack([
    test_predict_label[:, :11].argmax(1),
    test_predict_label[:, 11:22].argmax(1),
    test_predict_label[:, 22:33].argmax(1),
    test_predict_label[:, 33:44].argmax(1),
]).T

test_label_pred = []
for x in test_predict_label:
    test_label_pred.append(''.join(map(str, x[x!=10])))
    
import pandas as pd
df_submit = pd.read_csv('./data/mchar_sample_submit_A.csv')
df_submit['file_code'] = test_label_pred
df_submit.to_csv('./renset18_4.csv', index=None)

In [ ]:
df_submit.to_csv('./renset18_4.csv', index=None)